In [12]:
import yfinance as yf
import pandas as pd 


#Fetch MSFT stock data using yfinance
msft_yf = yf.download("MSFT", period="1y")

#Display the first few rows
print("MSFT Data from yfinance:")
print(msft_yf.head())

#Flatten the Yfinance DF to allow merge later on 
msft_yf.reset_index(inplace=True)
msft_yf['date'] = pd.to_datetime(msft_yf['Date']).dt.date


#Rename columns to ensure consistency
#msft_yf.rename(columns={'Date': 'date'}, inplace=True)




[*********************100%***********************]  1 of 1 completed

MSFT Data from yfinance:
Price            Close        High         Low        Open    Volume
Ticker            MSFT        MSFT        MSFT        MSFT      MSFT
Date                                                                
2024-05-17  417.012756  419.702159  414.849350  419.325045  15352200
2024-05-20  422.103760  423.522872  416.794460  417.012787  16272100
2024-05-21  425.775574  429.675664  421.617452  423.582367  21453300
2024-05-22  427.244324  429.119958  423.880133  426.817603  18073700
2024-05-23  423.751099  430.300887  422.183134  429.675676  17211700


In [ ]:
from alpha_vantage.timeseries import TimeSeries
import os 

#Initialise Alpha Vantage with my API key
ts = TimeSeries(key='0JH4IK0C0E6RFOB8', output_format='pandas')

#Get daily time series data for MSFT
data, meta_Data = ts.get_daily (symbol='MSFT', outputsize='full')
data.reset_index(inplace=True) # Make the 'date' column from index
data['date'] = pd.to_datetime(data['date']).dt.date

#Display the first few rows from Alpha Vantage
print("\nMSFT Financial Data from Alpha Vantage:") 
print(data.head())


print("\nData Types of yfinance data:")
print(msft_yf.dtypes)
print("\nData Types of Alpha Vantage data:")
print(data.dtypes)

print("\nFirst few rows of yfinance data:")
print(msft_yf.head())
print("\nFirst few rows of Alpha Vantage data:")
print(data.head())

#Inspect the data types and structure after processing
print("\nAfter Processing - YFinance Data Info:")
print(msft_yf.info())
print(msft_yf.head())

print("\nAfter Processing - Alpha Vantage Data Info:")
print(data.info())
print(data.head())


#Combine the information and save both datasets to CSV
# try:
#     print("\nCombined Data:")
#     print(combined.head())
#     combined = pd.merge(msft_yf, data, on='date', suffixes=('_yf', '_av'))
#     msft_yf.to_csv("MSFT_yfinance.csv", index=False)
#     data.to_csv("MSFT_alphavantage.csv", index=False)
#     combined.to_csv("MSFT_combined.csv", index=False)
# except Exception as e:
#     print("\nMergeError:", e)

    
#내일 이거해보깅 ㅎㅅㅎ 지겹구려

try:
    # Attempt to merge the data
    combined = pd.merge(msft_yf, data, on='date', suffixes=('_yf', '_av'))
    print("\nSuccessfully merged data!")
except Exception as e:
    print("\nMergeError:", e)
    combined = None  # Ensure the variable exists even if merge fails

# Check if combined exists and print
if combined is not None:
    print("\nCombined Data:")
    print(combined.head())

    # Save both datasets to CSV
    msft_yf.to_csv("MSFT_yfinance.csv", index=False)
    data.to_csv("MSFT_alphavantage.csv", index=False)
    combined.to_csv("MSFT_combined.csv", index=False)
else:
    print("Failed to create the combined DataFrame.")


ValueError: We have detected your API key as 0JH4IK0C0E6RFOB8 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.

From here, I'm going to:
1. plot the closing prices (Close or equivalent) from both yfinance and Alpha Vantage to compare trends over time.
2. calculate andvisualize the daily % change in stock prices to analyze volatility. 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(msft_yf['date'], msft_yf['Close'], label='YFinance Close Price',  alpha=0.7)
plt.plot(data['date'], data['4. close'], label='Alpha Vantage Close Price', alpha=0.7)
plt.title('MSFT Stock Price Trends')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show() 


#Daily % change
msft_yf['Daily Change (%)'] = msft_yf['Close'].pct_change() * 100
data['Daily Change (%)'] = data['4. close'].pct_change() * 100

plt.figure(figsize=(12, 6))
plt.plot(msft_yf['date'], msft_yf['Daily Change (%)'], label='YFinance Daily Change', alpha=0.7)
plt.plot(data['date'], data['Daily Change (%)'], label='Alpha Vantage Daily Change', alpha=0.7)
plt.title('Daily Percentage Change in MSFT Stock Price')
plt.xlabel('Date')
plt.ylabel('Daily Change (%)')
plt.legend()
plt.show()

#volume comparison
# plt.figure(figsize=(12, 6))
# plt.bar(msft_yf['date'], msft_yf['Volume'], label='YFinance Volume', alpha=0.7, width=0.5)
# plt.gcf().autofmt_xdate() # Automatically format x-axis dates
# plt.bar(data['date'], data['5. volume'], label='Alpha Vantage Volume', alpha=0.7, width=0.5)
# plt.title('MSFT Trading Volume Comparison')
# plt.xlabel('Date')
# plt.ylabel('Volume')
# plt.legend()
# plt.show()

#Moving Averages
msft_yf['20-Day MA'] = msft_yf['Close'].rolling(window=20).mean()
msft_yf['50-Day MA'] = msft_yf['Close'].rolling(window=50).mean()

plt.figure(figsize=(12, 6))
plt.plot(msft_yf['date'], msft_yf['Close'], label='Close Price', alpha=0.7)
plt.plot(msft_yf['date'], msft_yf['20-Day MA'], label='20-Day MA', linestyle='--')
plt.plot(msft_yf['date'], msft_yf['50-Day MA'], label='50-Day MA', linestyle='--')
plt.title('MSFT Stock Price with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

#correlation Heatmap
import seaborn as sns

try:
    print("Is 'combined' defined?", 'combined' in locals())
    print("\nCombined DataFrame preview:")
    print(combined.head())
except NameError:
    print("The 'combined' DataFrame is not defined in your current session.")

combined_corr = combined.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(combined_corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap of MSFT Data')
plt.show()

#Price Distribution 
plt.figure(figsize=(12, 6))
sns.histplot(msft_yf['Close'], bins=50, kde=True, label='YFinance Close Price', color='blue', alpha=0.6)
sns.histplot(data['4. close'], bins=50, kde=True, label='Alpha Vantage Close Price', color='orange', alpha=0.6)
plt.title('Distribution of MSFT Closing Prices')
plt.xlabel('Close Price')
plt.ylabel('Frequency')
plt.legend()
plt.show()

#Candle Stick Chart
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates

msft_yf['date_num'] = mdates.date2num(msft_yf['date'])
ohlc = msft_yf[['date_num', 'Open', 'High', 'Low', 'Close']]

fig, ax = plt.subplots(figsize=(12, 6))
candlestick_ohlc(ax, ohlc.values, width=0.6, colorup='green', colordown='red')
ax.xaxis_date()
ax.set_title('MSFT Candlestick Chart')
ax.set_xlabel('Date')
ax.set_ylabel('Price')
plt.show()


#Using Pandas AI
import os 
from pandasai import PandasAI
from pandasai.llm.openai import OpenAI

#Initialize Pandas AI with OpenAI LLM
llm = OpenAI()
pandas_ai = PandasAI(llm)

#Example query on yfinance data
response = pandas_ai.run(msft_yf, prompt="What is the average closing price")
print (response)

# Example query on Alpha Vantage data
response = pandas_ai.run(data, prompt="What is the highest trading volume?")
print(response)

# Example query on both data
response = pandas_ai.run(data, prompt="이 데이터를 기반으로 투자 수익을 높이기 위한 분석 영어 보고서 부탁해 도입부는 이목을 끌만한 문제제기로 시작해줘. 도입부 문제 제기 후 전체적인 현황과 액션 아이템을 표 형태로 요약해줘. 도입부 이후에는 '투자 수익 올리기'에 적합한 분석 차트 5개를 넣어줘. 차트 안 글자는 깨지니까 영어로 줘. 각 그래프 별로 눈에 띄는 분석 인사이트와 액션 아이템을 실무자 입장에서 구체적으로 적어줘. 설명만 하지 말고 액션 아이템과 연결지어서 서술해줘야 해. 실제 수치 언급도 포함해서. 마지막에는 추가로 해보면 좋을 만한 분석을 5개 정도 추천해줘. 다른 출처의 데이터와의 융합 분석까지 고려해서.")
print(response)